In [2]:
!pip install transformers accelerate datasets

yRequirement already satisfied: transformers in /usr/local/lib/python3.10/dist-packages (4.41.2)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x

In [3]:
# !pip install --upgrade huggingface_hub

In [4]:
from datasets import load_dataset
multinews = load_dataset("multi_news", split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [5]:
multinews.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
multi_news = multinews.train_test_split(test_size=.2)


In [8]:
prefix= 'summarize:'
def process_function(examples):
  inputs= [prefix+doc for doc in examples['document']]
  model_inputs = tokenizer(inputs, max_length=1024, truncation = True)
  labels = tokenizer(text = examples['summary'], max_length=128, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [9]:
tokenized_multi_news = multi_news.map(process_function, batched = True)


Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer= tokenizer, model= 't5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10,
    # predict_with_generate = True,
    fp16 = True,
    # push_to_hub = True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_multi_news['train'],
    eval_dataset = tokenized_multi_news['test'],
    data_collator = data_collator,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.818400
2,3.268800,2.750828
3,3.013900,2.722818


Epoch,Training Loss,Validation Loss
1,No log,2.818400
2,3.268800,2.750828
3,3.013900,2.722818
4,2.962900,2.707467
5,2.933400,2.695100
6,2.916000,2.688149
7,2.895400,2.682322
8,2.895200,2.678731
9,2.878600,2.676749
10,2.883700,2.675908


TrainOutput(global_step=4500, training_loss=2.960873725043403, metrics={'train_runtime': 3028.7762, 'train_samples_per_second': 14.848, 'train_steps_per_second': 1.486, 'total_flos': 1.217264162439168e+16, 'train_loss': 2.960873725043403, 'epoch': 10.0})

In [14]:
test ="""Cell Splitting: In areas with high user density, a cell can become overloaded with too many
users, leading to dropped calls and slow data speeds. In such cases, a cell can be split into
smaller cells, often referred to as microcells or picocells, to accommodate more users and
improve capacity.
Cell Sectorization: Each cell can be divided into sectors by using directional antennas.
Sectorization increases the capacity of a cell by allowing it to handle more calls and data traffic
in a specific direction, which is particularly useful in areas with high user demand.
Handset Power Control: To minimize interference and conserve battery life, cellular handsets
adjust their transmit power based on their proximity to the nearest base station. This ensures
that the handset uses the minimum amount of power necessary to maintain a reliable
connection.
Cell Planning: Cellular network planning involves optimizing the placement and configuration
of cell sites to provide seamless coverage and capacity. Factors like terrain, building structures,
and user density are considered during cell planning.
Frequency Bands: Different frequency bands are allocated for cellular communication by
regulatory authorities. These bands are divided into uplink (from mobile device to cell tower)
and downlink (from cell tower to mobile device) frequencies and are used to provide various
cellular services like voice and data.
Cellular Generations: The cellular concept has evolved through different generations, including
2G (GSM), 3G (UMTS), 4G (LTE), and 5G. Each generation brings advancements in
technology, data speeds, and network capacity while building upon the basic principles of the
cellular concept.
The cellular concept has revolutionized communication by enabling widespread and efficient
mobile networks. It has paved the way for the development of modern smartphones and the
delivery of various services, from voice calls and text messaging to high-speed internet access
and multimedia streaming.
Cell Area:
In mobile communication, the term "cell area" refers to the geographic coverage area served
by an individual cell within a cellular network. Cells are the basic building blocks of cellular
networks, and each cell is responsible for providing wireless coverage to a specific area. Here
are some key points related to cell areas in mobile communication:
Cell Size: Cell areas can vary in size depending on several factors, including the population
density of the area, the terrain, and the available frequency spectrum. Cells are designed to
provide adequate coverage and capacity for a given area.Cell Coverage: The primary purpose of a cell is to provide radio coverage to mobile devices
within its designated area. This coverage allows mobile phones and other wireless devices to
connect to the cellular network and communicate with each other.
Cell Overlap: In practice, cell areas are often designed to overlap with neighbouring cells. This
overlap ensures that there are no coverage gaps, and it allows for seamless handoffs as mobile
users move from one cell to another. Overlapping cells facilitate continuous connectivity as
users travel through the network.
Cell Shape: Cells are typically depicted as hexagons on network diagrams, which is an
idealized shape that minimizes overlap and maximizes coverage. However, in real-world
scenarios, cell shapes can vary based on factors like terrain and the placement of cell towers.
Cell Capacity: The capacity of a cell, which is the number of users and devices it can support
simultaneously, depends on factors like the frequency band used, the technology generation
(2G, 3G, 4G, 5G), and the density of users in the area.
Cell Splitting: In areas with high user demand, cells may be split into smaller cells (microcells
or picocells) to increase capacity and provide better service quality. Cell splitting is a technique
used to alleviate congestion and improve network performance.
Cell Planning: Network operators carefully plan and optimize the placement and configuration
of cell sites to ensure efficient coverage, minimize interference, and meet capacity
requirements. This process involves determining the location and height of cell towers and
selecting appropriate antenna types.
Frequency Reuse: To maximize the use of available radio spectrum, the cellular concept
employs frequency reuse. This means that the same frequency band can be reused in cells that
are sufficiently far apart to minimize interference.
Sectorization: Each cell can be divided into sectors using directional antennas. Sectorization
increases the capacity and efficiency of a cell by focusing coverage and capacity in specific
directions, where it is needed most.
Cell Identification: Each cell is assigned a unique identification number or code, such as a Cell
ID or CGI (Cell Global Identity), which helps mobile devices and the network identify and
distinguish between different cells.
Overall, the concept of cell areas is central to the design and operation of cellular networks,
allowing for efficient use of radio spectrum and the provision of mobile communication
services to a wide range of users across diverse geographic areas"""

In [17]:
input_ids = tokenizer(test, max_length=1024, truncation=True, return_tensors='pt').input_ids
input_ids = input_ids.to('cuda')

In [18]:
import torch
with torch.no_grad():
  if model.device.type == 'cuda':
    outputs = model.generate(input_ids, max_length=128, num_beams=5)
summary_ids = outputs[0].tolist()
summary = tokenizer.decode(summary_ids, skip_special_tokens=True)
print(summary)


. Cells are typically depicted as hexagons on network diagrams, which is an idealized shape that minimizes overlap and maximizes coverage. Cells are typically depicted as hexagons on network diagrams, which is an idealized shape that minimizes overlap and maximizes coverage. Cells are typically depicted as hexagons on network diagrams, which is an idealized shape that minimizes overlap and maximizes coverage. The concept of cell areas is central to the design and operation of cellular networks, allowing for efficient use of available radio spectrum, and


In [19]:
ref_summary = """The concept of cell areas is fundamental to the efficiency and functionality of cellular networks, enabling widespread mobile communication coverage. Cells vary in size based on factors like population density and terrain, ensuring they can adequately serve their designated areas while optimizing capacity. Each cell's primary function is to provide seamless radio coverage to mobile devices, facilitating uninterrupted connectivity and communication. Cells are strategically overlapped with neighboring cells to prevent coverage gaps and enable smooth handoffs as users move between areas. Typically depicted as hexagons on network diagrams, cells can be adapted in shape and size to suit different environmental conditions and user densities. Techniques like cell splitting, where larger cells are divided into smaller microcells or picocells, are employed in high-demand areas to enhance capacity and service quality. Overall, cell areas are pivotal in maximizing spectrum use and delivering reliable mobile services to diverse geographic regions."""

In [28]:
!pip install rouge

In [29]:
from rouge import Rouge
rouge = Rouge()
scores=rouge.get_scores(summary, ref_summary)
scores

[{'rouge-1': {'r': 0.2169811320754717,
   'p': 0.5897435897435898,
   'f': 0.31724137537788355},
  'rouge-2': {'r': 0.10344827586206896,
   'p': 0.3409090909090909,
   'f': 0.15873015515803038},
  'rouge-l': {'r': 0.2169811320754717,
   'p': 0.5897435897435898,
   'f': 0.31724137537788355}}]

In [30]:
trainer.save_model()

In [31]:
model.save_pretrained('summarizer')